<a href="https://colab.research.google.com/github/heidousl/GoogleAgentsCourse_Capstone_Project_Nov2025/blob/main/MarketMind_Concierge_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# MarketMind Concierge - Intelligent Personal Investment Assistant
#
# INSTRUCTIONS FOR COLAB/KAGGLE:
# 1. Install necessary packages by running this in a cell:
#    !pip install google-adk nest_asyncio
#
# 2. Set your Google Cloud/Gemini API Key in the environment or config.
#    (The code below assumes local simulation or existing auth for the demo logic)

import asyncio
import nest_asyncio
from typing import List, Dict, Any
from google.adk.agents import Agent, AgentTool
from google.adk.types import UserMessage, Artifact, Message
from google.adk.tools import Tool

# Apply nest_asyncio to allow nested event loops in Jupyter/Colab
nest_asyncio.apply()

# --- 1. Session & State Management ---

class SessionStore:
    """
    A simple in-memory store to persist user risk profiles across turns.
    In production, this would be replaced by Firestore or Redis.
    """
    def __init__(self):
        self._sessions: Dict[str, Dict[str, Any]] = {}

    def get_session(self, session_id: str) -> Dict[str, Any]:
        if session_id not in self._sessions:
            self._sessions[session_id] = {}
        return self._sessions[session_id]

    def update_session(self, session_id: str, key: str, value: Any):
        if session_id not in self._sessions:
            self._sessions[session_id] = {}
        self._sessions[session_id][key] = value

# Global instance for the demo
session_store = SessionStore()


# --- 2. Custom Tool Definition ---

class FinancialDataTool(Tool):
    """
    Simulates a connection to a financial API (like AlphaVantage or Yahoo Finance).
    Returns structured JSON data about a stock ticker.
    """
    def __init__(self, name: str = "financial_data_fetcher"):
        super().__init__(
            name=name,
            description="Fetches current financial metrics (price, volatility, sector, P/E) for a stock ticker."
        )

    async def execute(self, ticker: str) -> str:
        print(f"      [Tool: {self.name}] 📡 Connecting to market data for '{ticker}'...")

        # Mock Data / Simulation
        t = ticker.strip().upper()
        if t == "GOOG":
            data = {"ticker": "GOOG", "price": 175.50, "sector": "Technology", "volatility": "Moderate", "pe_ratio": 25.1, "ytd_return": "+18%"}
        elif t == "VTSAX":
            data = {"ticker": "VTSAX", "price": 115.20, "sector": "Diversified Index", "volatility": "Low", "pe_ratio": 19.5, "ytd_return": "+10%"}
        elif t == "TSLA":
            data = {"ticker": "TSLA", "price": 220.00, "sector": "Automotive/Tech", "volatility": "High", "pe_ratio": 60.5, "ytd_return": "-5%"}
        elif t == "AAPL":
            data = {"ticker": "AAPL", "price": 180.00, "sector": "Consumer Electronics", "volatility": "Moderate", "pe_ratio": 28.0, "ytd_return": "+15%"}
        else:
            return f"Error: Data not found for symbol '{t}'."

        return str(data)


# --- 3. Specialized Agents ---

class DataFetcherAgent(Agent):
    """
    Agent 1: The Researcher.
    Responsibility: Reliable retrieval of raw data. No opinions, just facts.
    """
    def __init__(self, name: str = "DataFetcherAgent"):
        super().__init__(
            name=name,
            instruction="You are a financial researcher. Your only job is to use the 'financial_data_fetcher' tool to get raw data for a ticker and return it as an Artifact.",
            tools=[FinancialDataTool()],
        )

    async def execute(self, message: Message) -> Message:
        print(f"   [{self.name}] Received request. Invoking tool...")
        ticker = message.parts[0].text.strip()

        # Call the tool
        tool_result = await self.tools[0].execute(ticker)

        if "Error" in tool_result:
             return Message(parts=[UserMessage(text=tool_result)])

        # Wrap result in an Artifact for the next agent
        return Message(
            parts=[
                Artifact(name="financial_metrics", data=tool_result),
                UserMessage(text=f"Raw metrics retrieved for {ticker}.")
            ]
        )


class RiskAnalyzerAgent(Agent):
    """
    Agent 2: The Analyst.
    Responsibility: Interpret raw data based on a specific User Goal.
    """
    def __init__(self, name: str = "RiskAnalyzerAgent"):
        super().__init__(
            name=name,
            instruction="You are a senior investment analyst. Analyze the provided raw financial data (Artifact) specifically against the User's Investment Goal. Output a recommendation (BUY, SELL, or HOLD) and a brief reasoning.",
        )

    async def execute(self, message: Message) -> Message:
        print(f"   [{self.name}] Synthesizing data against user goal...")

        # 1. Extract Inputs (Goal & Data)
        raw_data_artifact = next((p for p in message.parts if isinstance(p, Artifact)), None)
        user_goal_text = next((p.text for p in message.parts if isinstance(p, UserMessage)), "")

        if not raw_data_artifact:
            return Message(parts=[UserMessage(text="Error: Missing financial data for analysis.")])

        # 2. Simulate Reasoning (In a real app, the LLM does this automatically via the prompt)
        # We manually simulate the logic here to demonstrate the architecture without an active LLM API connection.
        data = eval(raw_data_artifact.data) # Unpack mock JSON string
        volatility = data.get("volatility")
        ticker = data.get("ticker")

        recommendation = "NEUTRAL"
        reasoning = "Insufficient data."

        # Logic for 'Aggressive Growth'
        if "aggressive" in user_goal_text.lower():
            if volatility in ["High", "Moderate"]:
                recommendation = "BUY"
                reasoning = f"Fits your aggressive profile. The {volatility} volatility implies higher upside potential suitable for growth."
            else:
                recommendation = "HOLD"
                reasoning = "This asset is too conservative (Low Volatility) for your aggressive targets."

        # Logic for 'Safety' / 'Preservation'
        elif "safety" in user_goal_text.lower() or "preservation" in user_goal_text.lower():
            if volatility == "Low":
                recommendation = "BUY"
                reasoning = "Perfect alignment with your safety goals. Low volatility ensures capital preservation."
            else:
                recommendation = "AVOID"
                reasoning = f"Too risky! {volatility} volatility endangers your capital preservation goal."

        # 3. Construct Final Output
        final_response = (
            f"**Analysis for {ticker}**\n"
            f"• Goal: {user_goal_text}\n"
            f"• Volatility: {volatility}\n"
            f"• **Recommendation: {recommendation}**\n"
            f"• Insight: {reasoning}"
        )

        return Message(parts=[UserMessage(text=final_response)])


# --- 4. The Orchestrator (Master Agent) ---

class InvestmentMasterAgent(Agent):
    """
    The Orchestrator.
    Responsibility: Manage Session State and delegate tasks to specialists.
    """
    def __init__(self, name: str = "InvestmentMasterAgent", agents: List[Agent] = None):
        # Wrap specialists as tools
        self.fetcher_tool = AgentTool(agent=agents[0], name="data_fetcher")
        self.analyzer_tool = AgentTool(agent=agents[1], name="risk_analyzer")

        super().__init__(
            name=name,
            instruction="Orchestrate the investment analysis workflow. Manage user session state.",
            tools=[self.fetcher_tool, self.analyzer_tool],
        )

    async def execute(self, message: Message, session_id: str = "default") -> Message:
        user_input = message.parts[0].text.strip()
        print(f"\n🤖 [{self.name}] Processing: '{user_input}' (Session: {session_id})")

        # A. STATE CHECK: Handle Goal Setting
        if user_input.lower().startswith("set goal:"):
            new_goal = user_input.split(":", 1)[1].strip()
            session_store.update_session(session_id, "investment_goal", new_goal)
            return Message(parts=[UserMessage(text=f"✅ Risk Profile Updated: '{new_goal}'")])

        # B. STATE CHECK: Retrieve Goal
        session = session_store.get_session(session_id)
        current_goal = session.get("investment_goal")

        if not current_goal:
            return Message(parts=[UserMessage(text="⚠️  Wait! I don't know your risk profile yet.\nPlease reply with: 'Set goal: [Aggressive Growth | Safety | Balanced]'")])

        # C. WORKFLOW: Hub-and-Spoke Delegation
        print(f"   [{self.name}] Context Loaded: Goal is '{current_goal}'")

        # Step 1: Delegate to DataFetcher
        print(f"   [{self.name}] Delegating to DataFetcherAgent...")
        fetch_response = await self.fetcher_tool.execute(query=user_input)

        # Extract the artifact from the fetcher's response
        data_artifact = next((p for p in fetch_response.parts if isinstance(p, Artifact)), None)

        if not data_artifact:
            return fetch_response # Propagate error if no data found

        # Step 2: Delegate to RiskAnalyzer (Passing Data + Goal)
        print(f"   [{self.name}] Delegating to RiskAnalyzerAgent with context...")

        # Create a new message bundle for the analyst containing both the goal text and the data artifact
        analysis_payload = Message(parts=[UserMessage(text=current_goal), data_artifact])
        final_insight = await self.analyzer_tool.execute(message=analysis_payload)

        return final_insight


# --- 5. Main Execution Demo ---

async def run_market_mind_demo():
    print("=============================================================")
    print("   MARKETMIND CONCIERGE: INITIALIZING MULTI-AGENT SYSTEM")
    print("=============================================================")

    # 1. Setup
    fetcher = DataFetcherAgent()
    analyzer = RiskAnalyzerAgent()
    master = InvestmentMasterAgent(agents=[fetcher, analyzer])

    user_id = "investor_001"

    # 2. Interaction Loop

    # Scenario A: User asks for a stock without setting a profile
    response = await master.execute(Message(parts=[UserMessage(text="GOOG")]), session_id=user_id)
    print(f"\n>> AGENT: {response.parts[0].text}\n")

    # Scenario B: User sets their profile
    response = await master.execute(Message(parts=[UserMessage(text="Set goal: Aggressive Growth")]), session_id=user_id)
    print(f"\n>> AGENT: {response.parts[0].text}\n")

    # Scenario C: User asks for GOOG again (Should result in a BUY for Aggressive)
    response = await master.execute(Message(parts=[UserMessage(text="GOOG")]), session_id=user_id)
    print(f"\n>> AGENT: {response.parts[0].text}\n")

    # Scenario D: User asks for VTSAX (Should result in HOLD/NEUTRAL for Aggressive)
    response = await master.execute(Message(parts=[UserMessage(text="VTSAX")]), session_id=user_id)
    print(f"\n>> AGENT: {response.parts[0].text}\n")

    # Scenario E: Change Goal to Safety
    print("-" * 30)
    print("   [System] User changes mind...")
    response = await master.execute(Message(parts=[UserMessage(text="Set goal: Safety")]), session_id=user_id)
    print(f"\n>> AGENT: {response.parts[0].text}\n")

    # Scenario F: User asks for VTSAX again (Should result in BUY for Safety)
    response = await master.execute(Message(parts=[UserMessage(text="VTSAX")]), session_id=user_id)
    print(f"\n>> AGENT: {response.parts[0].text}\n")


if __name__ == "__main__":
    asyncio.run(run_market_mind_demo())